In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.close_figures=False 

import os, sys
sys.path.append(os.path.abspath(os.path.join('../')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn import linear_model
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns 
from mlmodels.model import *
from mlmodels.linear_models import *
from mlmodels.treebased_models import *
from mlmodels.pca_selection import *
from mlmodels.lasso_selection import *
from helper_libraries.model_pipeline import *
from helper_libraries.forecasting_tools import *
from helper_libraries.preprocessing_tools import *
from tqdm.auto import tqdm
import datetime as dt
import sklearn.preprocessing
import sklearn.utils
import logging
import warnings

sns.set_context("paper", font_scale=1.7)
sns.set_style("ticks", {"axes.grid": True, "grid.color": "0.95", "grid.linestyle": "-"})

# Outline

* Load some sample data for testing
* Run tests on various classes and functions

# Prepare Logger

In [2]:
# Formatting for log messages
log_format = "[{asctime}] — [{module:15.15} - {funcName:12.12}] — [{levelname:<8s} — Line {lineno:4d}]: {message}"
formatter = logging.Formatter(fmt=log_format, datefmt="%Y-%m-%d %H:%M:%S", style="{")


# Add a StreamHandler and FileHandler that outputs nicely formatted messages
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
file_handler = logging.FileHandler("tests_postselection.log", mode="w")
file_handler.setFormatter(formatter)

# Logging level debug will output practically everything
logging.basicConfig(
    level=logging.DEBUG,
    format=log_format,
    datefmt="%Y-%m-%d %H:%M:%S",
    style="{",
    handlers=[stream_handler, file_handler],
)

# Will capture warnings and format them into the log
logging.captureWarnings(True)

# Name of logger is name of the script
logger = logging.getLogger(__name__)

# # This line is actually unnecssary if you only run this cell once
# # Otherwise, want to prevent duplicate handlers so clear existing ones
# logger.handlers.clear()
# logger.addHandler(stream_handler)
# logger.addHandler(file_handler)

# # Add a StreamHandler that deals with warnings
# logging.getLogger("py.warnings").handlers.clear()
# stream_handler = logging.StreamHandler()
# stream_handler.setFormatter(formatter)
# file_handler = logging.FileHandler("tests_postselection.log", mode = 'w')
# file_handler.setFormatter(formatter)

# logging.getLogger("py.warnings").addHandler(stream_handler)
# logging.getLogger("py.warnings").addHandler(file_handler)

# # Prevent logger from sending messages to the root ( will give duplicate messages )
# logger.propagate = False

# Load Data

In [3]:
# Set up some sample data for the tests
sample_data_df = pd.read_parquet('../../data/proc/_temp/1996_all.parquet')

# Load data
iris_df = sns.load_dataset('iris')
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# Tests - Post Selection

## Set up high-dimensional data

The original columns are X = (petal width, sepal length, and sepal width)
The new columns are 
$$ X * H + \varepsilon$$
where $H$ is a rotation matrix (invertible with probabiltiy one actualy) obtained from a $3 \times 3$ matrix consisting of $U(0,1)$ entires and $\varepsilon$ consists of entries from $U(-5, 5)$. 

Also define new dependent variable Y as a rotation of the true X variables. 

In [4]:
# Copy the iris data set and add a bunch of noisy columns based on
# rotations of the original columns plus noise
np.random.seed(1)
iris_hf_df = iris_df.copy()
K = 150
X_new_cols = []
for k in range(K):
    new_col_names = [f"petal_width_{k}", f"sepal_length_{k}", f"sepal_width_{k}"]
    iris_hf_df[new_col_names] = (
        iris_hf_df[["petal_width", "sepal_length", "sepal_width"]] @ np.random.rand(3, 3)
        + (np.random.rand(len(iris_hf_df), 3) - 0.5) * 30
    )
    X_new_cols.append(new_col_names)
    
iris_hf_df['petal_length'] = iris_hf_df[["petal_width", "sepal_length", "sepal_width"]] @ np.random.rand(3, 1)

X_new_cols = sum(X_new_cols, [])

In [5]:
# Set up INS/OOS sample data
np.random.seed(1)
ins_frac = 0.1
iris_ins_df = iris_hf_df.sample(70).reset_index(drop = True)
iris_oos_df = iris_hf_df.sample(70).reset_index(drop = True)

Y_ins = iris_ins_df[['petal_length']]
X_ins = iris_ins_df[X_new_cols]
Y_oos = iris_oos_df[['petal_length']]
X_oos = iris_oos_df[X_new_cols]

## Estimate
Here, I use [standard lasso] and compare with [pca -> lasso]

In [8]:
# Component algos
model_forecast_lasso = LASSO({'lambda': 1e-5, 'use_intercept': False, 'seed': 5}, n_iter = 12)
model_forecast_enet = ENet({'lambda': 1e-5,'l1_ratio': 0.5,'use_intercept': True, 'seed': 5}, n_iter = 14)
model_forecast_ols = LinearRegression({})
model_selection_pca = PCA_selection({})
model_selection_lasso = LASSO_selection({})
 
# Post-selection algos
model_pca_lasso = PostSelectionModel(model_selection_pca, model_forecast_lasso, n_iter = 58)
model_lasso_ols = PostSelectionModel(model_selection_lasso, model_forecast_ols)
model_pca_enet  = PostSelectionModel(model_selection_pca, model_forecast_enet)
model_list = [model_forecast_lasso, model_pca_lasso, model_lasso_ols, model_pca_enet]

# Split into train/validate for optimal hyperparams
mtrain = ModelTrainer(model_list, Y_ins, X_ins, seed = 444)
mtrain.validation(frac=0.5, n_iter_default = 100)

# Testing
mtest = ModelTester(mtrain)
model_forecasts, model_params = mtest.forecast(Y_oos, X_oos)

[2021-11-30 17:03:34] — [model_pipeline  - validation  ] — [INFO     — Line  118]: Validating Lasso
[2021-11-30 17:03:34] — [linear_models   - fit         ] — [WARNING  — Line  101]: LASSO Convergence Warning
[2021-11-30 17:03:34] — [linear_models   - fit         ] — [WARNING  — Line  101]: LASSO Convergence Warning
[2021-11-30 17:03:34] — [linear_models   - fit         ] — [WARNING  — Line  101]: LASSO Convergence Warning
[2021-11-30 17:03:34] — [linear_models   - fit         ] — [WARNING  — Line  101]: LASSO Convergence Warning
[2021-11-30 17:03:34] — [linear_models   - fit         ] — [WARNING  — Line  101]: LASSO Convergence Warning
[2021-11-30 17:03:34] — [model_pipeline  - validation  ] — [INFO     — Line  118]: Validating PostSelectionModel(PCA_selection -> Lasso)
[2021-11-30 17:03:35] — [model_pipeline  - validation  ] — [INFO     — Line  118]: Validating PostSelectionModel(Lasso_selection -> LR)
[2021-11-30 17:03:35] — [lasso_selection - fit         ] — [WARNING  — Line   58]:

## View results

In [ ]:
def view_results(model_index):
    
    results_df = pd.concat(
        [
            model_forecasts[model_index].rename(columns={"petal_length": "forecast"}),
            Y_oos.rename(columns={"petal_length": "true"})
        ],
        axis=1,
    )

    display(smf.ols('true ~ forecast - 1', data = results_df).fit().summary())

    fig, ax = plt.subplots(figsize = (5,5))
    sns.regplot(x = 'forecast', y = 'true', data = results_df, ax  = ax)

### Pure Lasso

In [ ]:
view_results(0)

### PCA Lasso

In [ ]:
view_results(1)

### Lasso OLS

In [ ]:
view_results(2)

### PCA Enet

In [ ]:
view_results(3)

### Do the estimates make sense?

In [ ]:
# Params for just Lasso
fig, ax = plt.subplots(figsize = (10,5))
plt.plot(model_params[0], marker = 'o')
ax.set_xlabel('Regressor #')
ax.set_ylabel('Lasso-estimated coefficient')
ax.set_title(f'Lasso estimates {np.sum(model_params[0] != 0)} non-zero coefficients');

In [ ]:
# Params for PCA then Lasso
display(model_params[1])
print()
print(f'PCA picks {np.sum(model_params[1]["selection"] != 0)} components');
print(f'... then Lasso estimates {np.sum(model_params[1]["forecast"] != 0)} non-zero coefficients');

In [ ]:
# Params for Lasso then OLS
# display(model_params[2])
print()
print(f'Lasso picks {np.sum(model_params[2]["selection"] != 0)} components');
print(f'... then OLS estimates {np.sum(model_params[2]["forecast"] != 0)} non-zero coefficients');

In [ ]:
# Params for PCA then Enet
display(model_params[3])
print()
print(f'PCA picks {np.sum(model_params[3]["selection"] != 0)} components');
print(f'... then Enet estimates {np.sum(model_params[3]["forecast"] != 0)} non-zero coefficients');